In [39]:
from llama_index.llms.groq import Groq

In [40]:
import os
from llama_index.core import Settings
#from llama_index.llms.openai import OpenAI
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.response.pprint_utils import pprint_response

In [41]:
from dotenv import load_dotenv
load_dotenv()

True

In [42]:
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")
llm = Groq(temperature=0,model="llama3-70b-8192")
Settings.llm = llm


In [108]:
# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("youtubetranscript/text/").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

In [109]:
query_engine = index.as_query_engine(streaming=True, similarity_top_k=1)
response_stream = query_engine.query(
    "give me two million subscriber video link",
)
response_stream.print_response_stream()

There is no information in the provided context about a video with 2 million subscribers. The context only mentions a video with 1 million subscribers, and the link provided is for that video: https://www.youtube.com/watch?v=nuInJo5U4QQ.

In [138]:
query_engine = index.as_chat_engine(chat_mode="context", llm=llm,verbose=True)
response = query_engine.chat("give me two million subscriber video link and give me publish_date as well")
print(response.response)

Based on the provided context, I can extract the required information for you.

The video link for the 1 million subscribers (not 2 million) is:
https://www.youtube.com/watch?v=1Hc6Ws2tn-g

The publish date for this video is:
2023-10-29 00:00:00


In [125]:
response = query_engine.chat("tell me about Teach KG at Home? video link and give me publish_date as well")
print(response)

You're asking about the video "Teach KG at Home" by Zeeshan Usmani!

Here's the information you requested:

**Video Link:** https://www.youtube.com/watch?v=i-c8oQ-vmFM
**Publish Date:** 2023-05-10 00:00:00

In this video, Zeeshan Usmani shares his thoughts on teaching KG (Kindergarten) at home, highlighting the importance of education and the challenges faced by the education system in Pakistan. He discusses the differences in curriculum and teaching methods between various boards and schools, and how this can affect a child's ability to adapt to different educational environments.

Zeeshan Usmani also shares his vision for a more unified and effective education system, where children can learn and grow without being limited by the constraints of a particular board or school.

I hope that helps!


In [133]:
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
query_engine = index.as_chat_engine(chat_mode="context", llm=llm,
                                    context_prompt=(
                                        "You are a chatbot, able to have normal interactions, as well as talk"
                                        " about an Zeeshan usmani youtube channel videos"
                                        "source, title, description, publish_date, author, transcript, youtubelink"
                                        "Here are the relevant documents for the context:\n"
                                        "{context_str}"
                                        "\nInstruction: Use the previous chat history, or the context above, to interact and help the user, and always return the youtube link and publish date from soruce document"
                                    )
                                    ,verbose=False,similarity_top_k=1)



In [137]:
response = query_engine.chat("tell me about Teach KG at Home and summary? video link and give me publish date as well")
print(response)
#pprint_response(response,show_source=True)

Based on the provided context, here's the information about the "Teach KG at Home" video:

**Video Title:** Teach KG at Home | گھر پر کے جی پڑھائیں | केजी को घर पर ही पढ़ाएं | Pre-KG | KG | home schooling

**Video Link:** https://www.youtube.com/watch?v=i-c8oQ-vmFM

**Publish Date:** 2023-05-10 00:00:00

**Summary:** In this video, the creator, Zeeshan Usmani, discusses the importance of homeschooling and teaching kindergarten (KG) at home. He shares his personal experience of teaching his own children at home and provides tips and resources for parents who want to take an active role in their child's education. He also discusses the benefits of homeschooling, including flexibility and personalized learning.

**Video Description:** Unknown

**View Count:** 118,024

**Length:** 444 seconds


In [136]:
response = query_engine.chat("How can i migrate from pakistan to dubai? give me youtube video link and publish date")
print(response)
pprint_response(response,show_source=True)

Based on the provided context, here's the information about migrating from Pakistan to Dubai:

**Video Link:** https://www.youtube.com/watch?v=JFQ5XQpQrOQ

**Publish Date:** 2022-07-24 00:00:00

This video is titled "How to Migrate from Pakistan to Dubai | Dubai Visa | Job Seeker Visa | Golden Visa" and has a view count of 234,119. The video is uploaded by ZUSMANI YOUTUBE CHANNEL and has a length of 14 minutes and 31 seconds.

In this video, the creator shares information on how to migrate from Pakistan to Dubai, including the requirements for a Job Seeker Visa and the Golden Visa, as well as tips for finding a job in Dubai.
Final Response: Based on the provided context, here's the information
about migrating from Pakistan to Dubai:  **Video Link:**
https://www.youtube.com/watch?v=JFQ5XQpQrOQ  **Publish Date:**
2022-07-24 00:00:00  This video is titled "How to Migrate from
Pakistan to Dubai | Dubai Visa | Job Seeker Visa | Golden Visa" and
has a view count of 234,119. The video is uplo

In [114]:
#query_engine = index.as_query_engine()
response = query_engine.chat("How many youtube subscribers of zishan usmani yotube channel? he told me that is his video he has 2 million subscribers?")
pprint_response(response,show_source=True)
print(response)

Final Response: I think there might be some confusion! According to
the transcript of one of Zeeshan Usmani's videos, he mentioned that he
has around 29,000 subscribers, not 2 million.  Here's the relevant
part from the transcript:  "...my total grade is B+, 29,000
subscribers, total video views are 12 million..."  So, as of the
publish date of that video, Zeeshan Usmani's YouTube channel had
around 29,000 subscribers.
______________________________________________________________________
Source Node 1/1
Node ID: fa8e5e48-37e9-43e6-a733-7883daac350a
Similarity: 0.8529550929354701
Text: Now no person has  Go and talk about any topic, he will have the
answer, firstly he will not say that I don't know, I don't know, you
can ask someone else, it is not my topic, he definitely has some
opinion to give, from cancer patient to cancer.  From how will Shifa
be in the marriage, she will know everything from the chief of the
country's ar...
I think there might be some confusion! According to the 

In [83]:
response = query_engine.chat("give me all youtube channel videos of zeeshan usmani in table foramt?")
#pprint_response(response,show_source=True)
print(response)

I apologize, but I don't have access to the entire list of Zeeshan Usmani's YouTube channel videos. The context provided only includes one video transcript.

However, I can provide you with a sample table format for Zeeshan Usmani's YouTube channel videos:

| **Video Title** | **Description** | **Publish Date** | **Author** | **Transcript** | **YouTube Link** |
| --- | --- | --- | --- | --- | --- |
| ریحان اللہ والاسے چبھتے سوالات | ... | ... | Zeeshan Usmani | ... | https://www.youtube.com/watch?v=bETjYxNIsLk |
| ... | ... | ... | Zeeshan Usmani | ... | ... |
| ... | ... | ... | Zeeshan Usmani | ... | ... |

Please note that I only have access to the one video transcript provided in the context. If you provide me with the entire dataset of Zeeshan Usmani's YouTube channel videos, I can help you populate this table.

Let me know if there's anything else I can assist you with!


In [54]:
response = query_engine.query("what is price of success?")
pprint_response(response,show_source=True)
print(response)

Final Response: According to the transcript, the price of success is
the costs that successful people pay, which are not visible, such as
hard work, sacrifices, and tiredness. They have to pay the price of
their success, which includes their time, relationships, and personal
life. They have to work hard, miss important events, and sacrifice
their personal life to achieve success.
______________________________________________________________________
Source Node 1/2
Node ID: 939e22b2-2bb9-4ab5-aacd-aee28d773aad
Similarity: 0.8374664317783173
Text: {     "source": "qvnjim6o8Ik",     "title": "Price of Success |
کامیابی کی قیمت",     "description": "Unknown",     "view_count":
18521,     "thumbnail_url":
"https://i.ytimg.com/vi/qvnjim6o8Ik/hq720.jpg?v=65f4c4d6",
"publish_date": "2022-06-15 00:00:00",     "length": 264,
"author": "Zeeshan Usmani",     "transcript": "Assalam alekum, m...
______________________________________________________________________
Source Node 2/2
Node ID: 2be2a327

In [72]:
response = query_engine.chat("give me the summary which was publish date on 2022-01-19?")
pprint_response(response,show_source=True)
print(response)

Final Response: I think you're referring to the video "Happy New Year
2022 | نیا سال مبارک" by Zeeshan Usmani.  Here's a summary of the
video:  Zeeshan Usmani starts by wishing his viewers a Happy New Year.
He then reflects on how people often focus on the negative aspects of
the world, such as corruption, oppression, and injustice. However, he
suggests that instead of complaining, we should take responsibility
for our own actions and make a positive impact.  He criticizes people
who borrow views from others, without having original ideas or taking
action. He also pokes fun at motivational speakers who use dramatic
gestures and expressions to engage their audience, but lack substance.
Zeeshan Usmani emphasizes the importance of self-reflection, taking
responsibility, and making a positive difference in the world.
**Publish Date:** 2021-12-20 (not 2022-01-19, I apologize for the
mistake)  Let me know if you'd like more information!
_______________________________________________________

In [73]:
response = query_engine.chat("this is wrong info there is video available which publisg date is 2022-01-19?")
pprint_response(response,show_source=True)
print(response)

Final Response: I apologize for the mistake. I don't have information
about a specific video published on 2022-01-19. The transcripts I have
access to are from two videos:  1. "Meeting with Mufti Tariq Masood |
مفتی طارق مسعود سے ملاقات | Zeeshan Usmani" 2. "What Pakistan Should
Do پاکستان کیا کرے | Zeeshan Usmani"  If you can provide more context
or information about the video published on 2022-01-19, I'll try to
help you find the summary.
______________________________________________________________________
Source Node 1/2
Node ID: e360f9c8-3733-4fbc-9569-781df30b6278
Similarity: 0.8317222791567985
Text: {     "source": "u0-krlmX1_0",     "title": "Meeting with Mufti
Tariq Masood | مفتی طارق مسعود سے ملاقات | Zeeshan Usmani",
"description": "Unknown",     "view_count": 389959,
"thumbnail_url":
"https://i.ytimg.com/vi/u0-krlmX1_0/hq720.jpg?v=65f4c4d6",
"publish_date": "2022-02-28 00:00:00",     "length": 952,
"author": "Zeeshan Us...
__________________________________________________

In [57]:
response = query_engine.query("tell me about the people of pakistan?")
pprint_response(response,show_source=True)
print(response)

Final Response: According to the given information, Pakistan has a
population of 219 million people. Out of these, 10 million people are
disabled, either handicapped or having some other kind of disability.
There are 40 million people who are below 14 years of age, and 10
million people who are above 65 years old. Additionally, there are
86000 Pakistanis who are imprisoned, 11000 Pakistanis who are
imprisoned in other countries, 160,000 Pakistanis who are suffering
from AIDS, and 17 million people who have Hepatitis B or C.
Furthermore, 32 million people in Pakistan have heart diseases, and
37.5 million people are diabetic.
______________________________________________________________________
Source Node 1/2
Node ID: 907f620b-c0e9-4f43-bb98-5c6915855c48
Similarity: 0.8677311262422863
Text: Pakistan Air Force, Pakistan Navy is a part of Pakistan Army,
which protects all these 219 million population, absolutely inside
Pakistan. According to a rough estimate, there are about 10 million
s